> Go is a *statically typed*, *garbage collected*, *compiled* language that is capable of producing highly concurrent, thread-safe programs that will scale as needed

*Static typing* 
- Each statement is checked at compile time.
- Data types are bound to variables (instead of to values as in dynamically typed languages like PHP) and must be defined when declaring variables

*Garbage collection*
- Developers don't need to explicitly allocate and deallocate memory for storing variables. Instead, developers use a garbage collector to monitor the program's memory usage.
- The garbage collector periodically determine which memory is still in use and which is not, and releases the memory no longer being used by the program
- A garbage collector makes it easier to write code but may have a performance cost; As you generate more memory usage(or "garbage"), the garbage collector need to spend time releasing that memory later. So, excessive garbage can lead to poorly performing programs.

*Compilation*
- One of the original design goals behind the Go language was fast compile times.
- The `go build` command is used to compile the source code.
- Errors in the code such as mismatch types are caught at compile time. unlike interpreted languages that will continue to execute every line till it encounters the error and raises and exception.

**Keywords Operators and Delimiters**
- keywords are reserved and cannot be used as an identifier.
- Follow these links to find the list of [keywords](https://go.dev/ref/spec#Keywords), [operators](https://go.dev/ref/spec#Operators) and [assignment operators](https://go.dev/ref/spec#assign_op)

### Numbers
Go has two types of numeric types; 
- *Architecture-independent* type which means regardless of the architecture you compile for, the type will the correct size, bytes, for your type.
- *Implementation-specific* type where the byte size of the numeric type can vary based on the architecture the program is built for.
- Architecture independent numeric types include; `uint8`, `uint16`, `uint32`, `uint64`, `int8`, `int16`, `int32`, `int64`, `float32`, `float64`, `complex64`, `complex128`, `byte`, `rune`.
- `byte` is alias for `uint8` and `rune` is alias for `int32`
- Architecture implementation specific types include; `uint`, `int`, `uintptr`
- size of `uint` will be 32 bits on 32-bit architecture and 64 bits on 64-bit architecture.
- list of these types and their ranges can be found [here](https://go.dev/ref/spec#Numeric_types)

Picking the correct type usually has more to do with performance for the target architecture you are programming for than the size of the data you are working with.
- For integer data, it is common to use the implementation types like `int` and `uint`. This typically results in the fastest processing speed for your target architecture
- If you know you wont exceed a specific size range, picking an architecture independent type can both increase speed and decrease memory usage.

*OverFlow & Wraparound* \
When performing mathematical calculations,  there is a potential to either "overflow" a number or "wraparound" a number.
- At compile time, if the compiler can determine a value will be too large to hold in the data type specified, it will throw an overflow error, indicating that the value calculated to store is too large for the data type you specified.


In [ ]:
// example of overflow

package main

import "fmt"

func main () {
	var maxUint uint8 = 260 // uint8's max value is 255
	fmt.Println("value: ", maxUint)
}

// this code will return an error at compile time

In [ ]:
// example of wraparound

package main

import "fmt"

func main () {
  var maxInt uint8 = 255 // uint8's max value is 255
  fmt.Println("value: ", maxInt+5)
}

// this code will compile successfully but the will print 4 instead of 260

- Go does not saturate variable during mathematical operations such as addition or multiplication. In languages that saturate, if you have a uint8 with a max value of 255 and add 1, the value is still the max(saturated) value of 255. In Go however, it always wraps around.
- Understanding the boundaries of your data helps you avoid potential bugs in your program in the future.
- read more about saturation arithmetic on [wikipedia](https://en.wikipedia.org/wiki/Saturation_arithmetic)

### Strings

Go uses double quotes and backticks for strings. \
Backticks create raw string literals. Double quotes create interpreted string literals.

- Any characters except newline and an unescaped double quotes may appear within the quotes.
- Backslashes (escape sequences) have no special meaning inside raw string literals.
- Raw string literals are useful for creating multiline templates and mock JSON data for testing, etc

### UTF-8
Go supports UTF-8 characters out of the box. [more on utf-8 on wikipedia](https://en.wikipedia.org/wiki/UTF-8)

In [ ]:
func main() {
	a := "Hello, 世界" // hello world (English + Chinese)
	fmt.Println(a)
}

### Runes
- A `rune` is an alias for int32 and is used to represent single characters. 
- A rune can be made up of 1, 2 or 3 int32 values, allowing for both single and multi-byte characters.
A rune can be defined using a single quote (') character

In [ ]:
func main () {
	a := 'A'

	fmt.Printf("%v (%T)\n", a, a) // outputs value of a and type of a.
}

// when printed as a value, a rune will be printed as an int32 (which is the UTF-8 encoding of the character)

**Iterating over UTF-8 characters**
- When iterating over a string and trying to access individual characters, we may encounter issues with UTF-8 characters

In [ ]:
func main () {
	a := "Hello, 世界" // 9 characters
	for i := 0; i < len(a); i++ {
		fmt.Printf("%d: %s\n", i, string(a[i]))
	}
}

// the output is unpredictable when iterating over UTF-8 string of characters.

Notice  that the unexpected characters in the output of the above code are printed for index 7-12. This is because multiple int32s make up the rune for the chinese characters but we are only taking part of it. \
In simpler terms; \
The issue is that the string contains both ASCII and UTF-8 characters. ASCII characters are represented by 1 byte each in Go. UTF-8 characters, on the other hand, are represented by multiple bytes (e.g., 3 bytes for Chinese characters like 世 or 界).
The len() function in the for loop simply counts the number of bytes, in the argument passed to it. This means the loop breaks the string into 1-byte pieces, regardless of whether the character is ASCII or UTF-8.
The string() function, much like len(), tries to interpret each byte individually as a single character (a 32-bit rune). This works fine for 1-byte ASCII characters, but it cannot correctly interpret UTF-8 characters because they are split into their individual bytes. As a result, the Chinese characters are broken into 3 invalid pieces each and printed as unexpected or garbled characters.

When iterating through each character in a string in Go, the proper way is to use the range keyword in the loop;

In [ ]:
func main () {
	a := "Hello, 世界" // 9 characters
  for i, c := range a {
    fmt.Printf("%d: %s \n", i, string(c))
  }
}

// the range keyword ensures that we use the proper index and length of int32s to capture the proper rune value

### Variables
- declaration: `var <name> <type>`
  - `var i int`
  - `var b bool`
- assignment: `<name> = <value>`
  - `i = 26`
  - `b = false`

**Zero Values**
- Variables when declared without values, assume a zero value of the same type as the variable. Complex types such as a `struct` have a zero value that is composed of the type's individual fields' zero values. 
- Certain types in Go such as maps, interfaces and pointers have no obvious zero values, hence their zero values is `nil`.

// zero value cheatsheet

```go
  var s string // ""
  var r rune // 0
  var bt byte // 0
  var i int // 0
  var ui uint // 0
  var c complex64 // 0
  var b bool // false
  var arr [2]int // [0 0]
  var obj struct {
    b bool
    arr [2]int
  } // {false [0 0]}
  var si []int // nil
  var ch chan string // nil
  var mp map[int]string // nil
  var fn func() // nil
  var ptr *string // nil
  var all any // nil
```

- When declaring a variable, we often want to initialize it with a value; `var f float64 = 3.14`
- When initializing a variable at declaration time,  we can use the `:=` operator; `i := 42`
- When using the `:=` operator, go infers the type of the variable from the value it is being assigned.
- For most types, inferring the type is no trouble. For numbers, the compiler has to guess the type, and by default, assumes that the variable is of type int or float64 (if there is a decimal point in it)
- If for numbers you desire some other type other that int or float64,  use the `var <name> <type> [= <value>]` to declare the variable or use variable casting like so; `i := uint32(42)`, `f := float32(3.14)`

Assigning multiple values